In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

path = './input/'

start_time = time.time()

train = pd.read_csv(path+"ct-sample_train_10000.csv")
train.columns = ['timestamp', 'sdk', 'platform', 'pub_idx', 'pub_app', 'pub_tagid', 'country', 'shop_idx', 'campaign_group_idx', 'campaign_idx', 'ads_idx', 'format_idx', 'ads_width', 'ads_height', 'adsize', 'bid', 'win', 'click', 'ctr']
test = pd.read_csv(path+"ct-sample_test.csv")

print('[{}] Finished to load data'.format(time.time() - start_time))

[0.0217151641846] Finished to load data


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print len(train)
print len(train.columns)

print train.dtypes.unique()
print train.dtypes

9999
19
[dtype('int64') dtype('O') dtype('float64')]
timestamp               int64
sdk                    object
platform               object
pub_idx                 int64
pub_app                object
pub_tagid              object
country                object
shop_idx                int64
campaign_group_idx      int64
campaign_idx            int64
ads_idx                 int64
format_idx              int64
ads_width               int64
ads_height              int64
adsize                 object
bid                     int64
win                     int64
click                   int64
ctr                   float64
dtype: object


In [3]:
import category_encoders as ce

# Get a new clean dataframe
obj_df = train.select_dtypes(include=['object']).copy()

# Specify the columns to encode then fit and transform
encoder = ce.backward_difference.BackwardDifferenceEncoder(cols=["pub_tagid"])
encoder.fit(obj_df, verbose=1)

# Only display the first 8 columns for brevity
encoder.transform(obj_df).iloc[:,0:7].head()

,col_pub_tagid_0,col_pub_tagid_1,col_pub_tagid_2,col_pub_tagid_3,col_pub_tagid_4,col_pub_tagid_5,col_pub_tagid_6
0,1.0,-0.998339,-0.996678,-0.995017,-0.993355,-0.991694,-0.990033
1,1.0,-0.998339,-0.996678,-0.995017,-0.993355,-0.991694,-0.990033
2,1.0,-0.998339,-0.996678,-0.995017,-0.993355,-0.991694,-0.990033
3,1.0,-0.998339,-0.996678,-0.995017,-0.993355,-0.991694,-0.990033
4,1.0,-0.998339,-0.996678,-0.995017,-0.993355,-0.991694,-0.990033


In [4]:
train.isnull().any().sum()
# isnull() 각 데이타의 null여부 True/False
# any() 한개라도 존재할경우 True
# sum() 갯수구하기
# null값이 존재하는 컬룸 수는 몇개인가?
train.sample(n=10)

,timestamp,sdk,platform,pub_idx,pub_app,pub_tagid,country,shop_idx,campaign_group_idx,campaign_idx,ads_idx,format_idx,ads_width,ads_height,adsize,bid,win,click,ctr
1788,1514732400,app,android,21,C190DAA7CEE67102FC3CDA9A8DB34E70,NaN,KOR,95,695,3254,53689,1,320,50,320x50,0,12,0,0.0
1963,1514732400,app,android,21,CFA421B6C06A85A95E105A16EA7CC1C1,NaN,KOR,95,695,3256,53678,1,320,50,320x50,58,0,0,0.0
2158,1514732400,app,android,21,F7EA140CA4C24CC52064F8329986FB44,NaN,KOR,95,695,3256,53681,1,320,50,320x50,1,0,0,0.0
4728,1514732400,app,android,34,cn.wps.moffice_eng,7ac6706421af4b7fa3e39e231dd98db2,KOR,170,645,2955,50884,14,0,0,0x0,1,0,0,0.0
5107,1514732400,app,android,34,com.apalon.weatherlive.free,691aaaeab875466fa24f6494dd82a0e8,KOR,170,647,2951,51516,11,320,480,320x480,1,0,0,0.0
2776,1514732400,app,android,32,com.gto.zero.zboost,130326099,KOR,95,694,3251,53643,4,300,250,300x250,4,2,0,0.0
8837,1514732400,app,android,34,com.eyefilter.night,a26b7a51292e42d2aa4ac77d4d5e4fde,KOR,82,696,3257,53721,4,300,250,300x250,5,2,0,0.0
1488,1514732400,app,android,21,A245A7F15D2E14F70B8EA5FD9C6141DA,NaN,KOR,170,646,2953,50909,1,320,50,320x50,3,0,0,0.0
6456,1514732400,app,android,34,com.camerasideas.instashot,0cb2dedea01b432b85869594f9c62f06,KOR,95,694,3252,53654,11,320,480,320x480,6,0,0,0.0
3247,1514732400,app,android,32,com.jb.security,130326135,KOR,231,670,3164,53492,1,320,50,320x50,30,17,0,0.0


In [5]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
train["sdk_code"] = lb_make.fit_transform(train["sdk"])
train["platform_code"] = lb_make.fit_transform(train["platform"])
train["pub_app_code"] = lb_make.fit_transform(train["pub_app"])
train["pub_tagid_code"] = lb_make.fit_transform(train["pub_tagid"])
train["country_code"] = lb_make.fit_transform(train["country"])
train["adsize_code"] = lb_make.fit_transform(train["adsize"])
train[["pub_app", "pub_app_code", "adsize", "adsize_code", 
      "pub_tagid", "pub_tagid_code", "sdk", "sdk_code"]]

del train['sdk']
del train['platform']
del train['pub_app']
del train['pub_tagid']
del train['country']
del train['adsize']

print(train.dtypes)

timestamp               int64
pub_idx                 int64
shop_idx                int64
campaign_group_idx      int64
campaign_idx            int64
ads_idx                 int64
format_idx              int64
ads_width               int64
ads_height              int64
bid                     int64
win                     int64
click                   int64
ctr                   float64
sdk_code                int64
platform_code           int64
pub_app_code            int64
pub_tagid_code          int64
country_code            int64
adsize_code             int64
dtype: object


/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [6]:
test["sdk_code"] = lb_make.fit_transform(test["sdk"])
test["platform_code"] = lb_make.fit_transform(test["platform"])
test["pub_app_code"] = lb_make.fit_transform(test["pub_app"])
test["pub_tagid_code"] = lb_make.fit_transform(test["pub_tagid"])
test["country_code"] = lb_make.fit_transform(test["country"])
test["adsize_code"] = lb_make.fit_transform(test["adsize"])
test[["pub_app", "pub_app_code", "adsize", "adsize_code", 
      "pub_tagid", "pub_tagid_code", "sdk", "sdk_code"]]

del test['sdk']
del test['platform']
del test['pub_app']
del test['pub_tagid']
del test['country']
del test['adsize']

print test.dtypes


timestamp               int64
pub_idx                 int64
shop_idx                int64
campaign_group_idx      int64
campaign_idx            int64
ads_idx                 int64
format_idx              int64
ads_width               int64
ads_height              int64
bid                     int64
win                     int64
click                   int64
ctr                   float64
sdk_code                int64
platform_code           int64
pub_app_code            int64
pub_tagid_code          int64
country_code            int64
adsize_code             int64
dtype: object


In [7]:
sub = pd.DataFrame()

sub['ads_idx'] = test['ads_idx']
sub['ctr'] = test['ctr']
# test.drop('ctr', axis=1, inplace=True)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[4.20506811142] Start XGBoost Training


In [8]:
y = train['ctr']


x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[4.21699309349] Start XGBoost Training


In [9]:
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

print watchlist

[(<xgboost.core.DMatrix object at 0x7f0d55ee3cd0>, 'train'), (<xgboost.core.DMatrix object at 0x7f0d55ee3f90>, 'valid')]


In [10]:
params = {'eta': 0.1, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'reg:linear', 
          'eval_metric': 'auc', 
          'random_state': 99, 
          'silent': True}

model = xgb.train(params, xgb.DMatrix(x1, y1), 260, watchlist, maximize=True, verbose_eval=10)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[0]	train-auc:1.42834	valid-auc:1.4782
[10]	train-auc:1.43148	valid-auc:1.48418
[20]	train-auc:1.43355	valid-auc:1.48574
[30]	train-auc:1.39768	valid-auc:1.45174
[40]	train-auc:1.39512	valid-auc:1.43912
[50]	train-auc:1.3936	valid-auc:1.43912
[60]	train-auc:1.40102	valid-auc:1.42874
[70]	train-auc:1.38784	valid-auc:1.42606
[80]	train-auc:1.38057	valid-auc:1.42101
[90]	train-auc:1.36735	valid-auc:1.41746
[100]	train-auc:1.36439	valid-auc:1.41916
[110]	train-auc:1.3743	valid-auc:1.41952
[120]	train-auc:1.3726	valid-auc:1.41952
[130]	train-auc:1.37725	valid-auc:1.42017
[140]	train-auc:1.37639	valid-auc:1.42192
[150]	train-auc:1.37858	valid-auc:1.42169
[160]	train-auc:1.38275	valid-auc:1.42031
[170]	train-auc:1.37945	valid-auc:1.42014
[180]	train-auc:1.38259	valid-auc:1.4204
[190]	train-auc:1.38455	valid-auc:1.42022
[200]	train-auc:1.38532	valid-auc:1.42072
[210]	train-auc:1.38346	valid-auc:1.41999
[220]	train-auc:1.38554	valid-auc:1.42031
[230]	train-auc:1.38804	valid-auc:1.42026
[240]	tr

In [11]:
test

,timestamp,pub_idx,shop_idx,campaign_group_idx,campaign_idx,ads_idx,format_idx,ads_width,ads_height,bid,win,click,ctr,sdk_code,platform_code,pub_app_code,pub_tagid_code,country_code,adsize_code
0,1520906400,135,95,735,3434,57222,1,320,50,1,0,0,0.0,0,0,0,0,0,1
1,1520906400,135,95,735,3434,57226,1,320,50,1,1,0,0.0,0,0,0,0,0,1
2,1520906400,135,170,731,3418,56376,1,320,50,1,0,0,0.0,0,0,0,0,0,1
3,1520906400,135,327,733,3427,57071,1,320,50,1,1,0,0.0,0,0,0,0,0,1
4,1520906400,135,327,733,3427,57091,1,320,50,1,1,0,0.0,0,0,0,0,0,1
5,1520906400,135,95,734,3432,57249,14,0,0,1,1,0,0.0,0,0,1,1,0,0
6,1520906400,135,170,731,3418,56375,14,0,0,2,1,0,0.0,0,0,1,1,0,0
7,1520906400,135,170,731,3418,56379,14,0,0,1,0,0,0.0,0,0,1,1,0,0
8,1520906400,135,170,731,3418,56383,14,0,0,1,0,0,0.0,0,0,1,1,0,0
9,1520906400,135,278,732,3421,57174,14,0,0,1,1,0,0.0,0,0,1,1,0,0


In [12]:
sub

,ads_idx,ctr
0,57222,0.0
1,57226,0.0
2,56376,0.0
3,57071,0.0
4,57091,0.0
5,57249,0.0
6,56375,0.0
7,56379,0.0
8,56383,0.0
9,57174,0.0


In [13]:
sub['ctr'] = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
sub.to_csv('./output/ct-sample_result.csv',index=False)

In [14]:
sub

,ads_idx,ctr
0,57222,0.465938
1,57226,0.287720
2,56376,0.472390
3,57071,0.267262
4,57091,0.267262
5,57249,0.879533
6,56375,0.281708
7,56379,0.528641
8,56383,0.528641
9,57174,0.851028
